In [1]:
# Start by importing the TELL package and information about your operating system:
import os 
import tell

import numpy as np
import pandas as pd

from glob import glob
from os import listdir


In [2]:
data_input_dir = '/Users/burl878/Documents/tell/tell/tell_data/sample_forcing_data/future_weather/rcp85hotter_ssp5/'

def find_csv_filenames(path_to_dir, suffix=".csv"):
    filenames = listdir(path_to_dir)
    return [filename for filename in filenames if filename.endswith( suffix )]

input_files = find_csv_filenames(data_input_dir)

for file in range(len(input_files)):
    filename = input_files[file]
    ba_code = filename.split('_', 1)[0]
    year = int(filename.split('_', 5)[5].strip('.csv'))
    
    # Read in the .csv file:
    df = pd.read_csv(os.path.join(data_input_dir, input_files[file]))                               
    
    # Convert the time to a datetime variable:
    df['datetime'] = pd.to_datetime(df['Time_UTC'])
    
    # Extract the year, month, day, and hour for each date:
    df['Year'] = df['datetime'].dt.strftime('%Y')
    df['Month'] = df['datetime'].dt.strftime('%m')
    df['Day'] = df['datetime'].dt.strftime('%d')
    df['Hour'] = df['datetime'].dt.strftime('%H')
    
    # Read in the future population for that BA and year combo:
    pop = tell.extract_future_ba_population(year, ba_code, 'ssp5', '/Users/burl878/Documents/tell/tell/tell_data')
    
    # Merge the two dataframes together using the year, month, day, and hour:
    df = df.merge(pop, on=['Year', 'Month', 'Day', 'Hour'])
    
    # Drop out the columns we don't need anymore:
    df.drop(['datetime', 'Year', 'Month', 'Day', 'Hour'], axis=1, inplace=True)
    
    # Write the output to the original .csv file:
    df.to_csv(os.path.join(data_input_dir, input_files[file]), index=False)
    
    del filename, ba_code, year, df